In [1]:
import pymc3
import umap
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import glob
import time
import os
from scipy.stats import zscore
import threading
import pickle
%matplotlib inline

## Load in z-scored data

In [2]:
df = pickle.load(open("fc_z.p","rb"))
df.head()

,Cell Size,Cell Circularity,Cell Aspect Ratio,Cell Tracker Intensity,PI Intensity,AnexinV Intensity
0_2G3,62.4662,-9.764209,2.257279,-0.613713,-0.912228,-1.053166
1_2G3,-0.327282,-9.343066,84.071640,-0.941838,-0.915353,-1.213429
2_2G3,-0.590511,-7.180160,15.145722,-0.891318,-0.914065,-1.198231
3_2G3,-0.531419,-8.459477,30.618888,-0.818131,-0.911537,-1.199510
4_2G3,-0.644231,0.652443,1.079209,-0.827969,-0.913367,-1.194802


## Run umap

In [3]:
# df_short = df.iloc[:100000,]
# df_short

In [4]:
# t1 = time.time()
# reducer = umap.UMAP()
# embedding = reducer.fit_transform(df_short)
# embedding.shape
# t2 = time.time()
# print(f"Time in seconds:{(t2-t1)/60}")
# with open("time_took.txt","w") as f:
#     f.write(str(t2-t1))

In [5]:
def run_umap(df,f_save):
    #df = pickle.load(open(in_f,"rb"))
    t1 = time.time()
    reducer = umap.UMAP()
    embedding = reducer.fit_transform(df.iloc[:,:-1])
    embedding.shape
    t2 = time.time()
    print(f"Time in minutes:{(t2-t1)/60}")

    pickle.dump(embedding,open(f_save,'wb'))
    with open(f_save + "_time_minutes.txt","w") as f:
        f.write(str((t2-t1)/60))
    return

In [6]:
maximumNumberOfThreads = 32
threadLimiter = threading.BoundedSemaphore(maximumNumberOfThreads)

class EncodeThread(threading.Thread):
    def run(self):
        threadLimiter.acquire()
        try:
            run_umap(df=df,f_save="umap_results.p")
        finally:
            threadLimiter.release()

In [7]:
EncodeThread().run()

/home/isshamie/software/anaconda2/envs/umap/lib/python3.7/site-packages/numba/compiler.py:602: NumbaPerformanceWarning: 
The keyword argument 'parallel=True' was specified but no transformation for parallel execution was possible.

To find out why, try turning on parallel diagnostics, see http://numba.pydata.org/numba-doc/latest/user/parallel.html#diagnostics for help.

File "../../../home/isshamie/software/anaconda2/envs/umap/lib/python3.7/site-packages/umap/rp_tree.py", line 135:
@numba.njit(fastmath=True, nogil=True, parallel=True)
def euclidean_random_projection_split(data, indices, rng_state):
^

  self.func_ir.loc))
/home/isshamie/software/anaconda2/envs/umap/lib/python3.7/site-packages/umap/nndescent.py:92: NumbaPerformanceWarning: 
The keyword argument 'parallel=True' was specified but no transformation for parallel execution was possible.

To find out why, try turning on parallel diagnostics, see http://numba.pydata.org/numba-doc/latest/user/parallel.html#diagnostics for help.

Time in minutes:1238.94629402558


## Save values

In [9]:
embedding = pickle.load(open("umap_results.p","rb"))
embedding


array([[ 14.301497 ,  -5.806734 ],
       [ -0.9557591, -15.899615 ],
       [ 11.957989 ,   9.023507 ],
       ...,
       [ -1.963811 ,  -0.5268107],
       [  9.15024  ,  -1.1823806],
       [ -9.841979 ,   0.8918857]], dtype=float32)

In [10]:
embedding.shape

(25577941, 2)